In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
task_names = [
#     "COLA", 
#     "SST2",
#     "MNLI",
    "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]


In [5]:
SEED = 1

## Load previously trained model
Hint: make sure the `bert_model` is initialized correctly!

In [6]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
#     freeze_bert=True,
    "bert_model": 'bert-large-cased',
    "max_len": 100   
}

In [7]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# Create tasks and payloads
tasks, payloads = create_tasks_and_payloads(task_names, **task_kwargs)

Using random seed: 252412
Loading RTE Dataset



CPU times: user 27.3 s, sys: 9.06 s, total: 36.3 s
Wall time: 31.1 s


In [8]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.0)],
 [Payload(RTE_train: tasks=[RTE], split=train),
  Payload(RTE_valid: tasks=[RTE], split=valid),
  Payload(RTE_test: tasks=[RTE], split=test)])

In [11]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

In [12]:
import os
import torch
model_dir = '/dfs/scratch0/chami/metal/logs/2019_03_17/RTE_23_22_34'
model_path = os.path.join(model_dir, 'best_model.pth')
device = torch.device(f"cuda:0")
model.load_state_dict(torch.load(model_path, map_location=device)["model"], strict=False)

#### Sanity check that task head is trained!

In [13]:
model.score(payloads[1])

{'RTE/RTE_valid/accuracy': 0.8483754512635379}

## Define slices for evaluation

In [14]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
#    "COLA": ["ends_with_question_mark"]
    "RTE": ["dash_semicolon", "more_people"]
}

# Create tasks and payloads
task_kwargs.update({"slice_dict": slice_dict})
_, payloads_slice = create_tasks_and_payloads(
    task_names, **task_kwargs
)

Using random seed: 128037
Loading RTE Dataset



Added label_set with 2490 labels for task RTE:dash_semicolon to payload RTE_train.
Added label_set with 2490 labels for task RTE:more_people to payload RTE_train.
Added label_set with 277 labels for task RTE:dash_semicolon to payload RTE_valid.
Added label_set with 277 labels for task RTE:more_people to payload RTE_valid.
Added label_set with 3000 labels for task RTE:dash_semicolon to payload RTE_test.
Added label_set with 3000 labels for task RTE:more_people to payload RTE_test.
CPU times: user 20.6 s, sys: 2.48 s, total: 23.1 s
Wall time: 24 s


In [15]:
payloads_slice

[Payload(RTE_train: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=train),
 Payload(RTE_valid: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=valid),
 Payload(RTE_test: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=test)]

#### Sanity check the number of examples in the train set

In [16]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [17]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

RTE 2490
RTE:dash_semicolon 945
RTE:more_people 64


### Evaluate baseline model on the slice of interest

In [18]:
model.score(payloads_slice[1])

{'RTE/RTE_valid/accuracy': 0.8483754512635379,
 'RTE:dash_semicolon/RTE_valid/accuracy': 0.7962962962962963,
 'RTE:more_people/RTE_valid/accuracy': 0.8333333333333334}

## Finetune model on slice of interest

In [19]:
from metal.mmtl.metal_model import MetalModel

from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

Uncomment below to only finetune on the slices, not the original task


In [20]:
# # only finetune on the slices, not the original task
# for p in payloads_slice:
#     p.task_names.remove('COLA')
# payloads_slice

NOTE: We are training on a different set of payloads than we initialized the model with.

In [21]:
trainer.train_model(
    model,
    payloads_slice,
    checkpoint_metric="RTE/RTE_valid/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=3,
    progress_bar=True,
    checkpoint_tasks=True,
    checkpoint_cleanup=False
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/mccreery/repos/metal/logs/2019_03_21/17_38_59/config.json


/dfs/scratch0/mccreery/repos/metal/metal/mmtl/trainer.py:579: UserWarning: checkpoint_tasks setting does not have the same thorough error checking that the normal checkpoint operation has, so you may accidentally be trying to checkpoint metrics that aren't going to be found in the metrics_dict if you're not careful.
  warnings.warn(msg)


/dfs/scratch0/mccreery/repos/metal/metal/mmtl/trainer.py:490: UserWarning: You requested glue score but have fewer than 9 tasks. Be aware.
  warnings.warn(msg)


[0.10 epo]: RTE:[train/loss=3.93e-01, RTE_valid/accuracy=8.19e-01] RTE:dash_semicolon:[train/loss=3.54e-01, RTE_valid/accuracy=7.59e-01] RTE:more_people:[train/loss=1.07e+00, RTE_valid/accuracy=6.67e-01] model:[train/loss=3.95e-01, train/lr=5.00e-05, valid/glue=8.19e-01]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/accuracy=0.819
Saving model at iteration 0.10 with best (max) score RTE:dash_semicolon/RTE_valid/accuracy=0.759
Saving model at iteration 0.10 with best (max) score RTE:more_people/RTE_valid/accuracy=0.667
[0.21 epo]: RTE:[train/loss=3.78e-01, RTE_valid/accuracy=8.23e-01] RTE:dash_semicolon:[train/loss=3.65e-01, RTE_valid/accuracy=7.59e-01] RTE:more_people:[train/loss=4.78e-01, RTE_valid/accuracy=8.33e-01] model:[train/loss=3.78e-01, train/lr=5.00e-05, valid/glue=8.23e-01]
Saving model at iteration 0.21 with best (max) score RTE/RTE_valid/accuracy=0.823
Saving model at iteration 0.21 with best (max) score RTE:more_people/RTE_valid/accuracy=0.833
[0.31 e

[1.03 epo]: RTE:[train/loss=3.22e-01, RTE_valid/accuracy=8.23e-01] RTE:dash_semicolon:[train/loss=2.47e-01, RTE_valid/accuracy=7.59e-01] RTE:more_people:[train/loss=1.29e-02, RTE_valid/accuracy=7.50e-01] model:[train/loss=2.95e-01, train/lr=5.00e-05, valid/glue=8.23e-01]
[1.13 epo]: RTE:[train/loss=2.18e-01, RTE_valid/accuracy=8.41e-01] RTE:dash_semicolon:[train/loss=1.04e-01, RTE_valid/accuracy=7.87e-01] RTE:more_people:[train/loss=1.38e+00, RTE_valid/accuracy=9.17e-01] model:[train/loss=2.01e-01, train/lr=5.00e-05, valid/glue=8.41e-01]
Saving model at iteration 1.13 with best (max) score RTE/RTE_valid/accuracy=0.841
Saving model at iteration 1.13 with best (max) score RTE:more_people/RTE_valid/accuracy=0.917
[1.23 epo]: RTE:[train/loss=2.27e-01, RTE_valid/accuracy=8.12e-01] RTE:dash_semicolon:[train/loss=1.59e-01, RTE_valid/accuracy=7.50e-01] RTE:more_people:[train/loss=2.30e-03, RTE_valid/accuracy=9.17e-01] model:[train/loss=2.05e-01, train/lr=5.00e-05, valid/glue=8.12e-01]
[1.33 ep

[2.05 epo]: RTE:[train/loss=3.99e-01, RTE_valid/accuracy=7.94e-01] RTE:dash_semicolon:[train/loss=3.10e-01, RTE_valid/accuracy=7.13e-01] RTE:more_people:[train/loss=1.82e-02, RTE_valid/accuracy=7.50e-01] model:[train/loss=3.74e-01, train/lr=5.00e-05, valid/glue=7.94e-01]
[2.15 epo]: RTE:[train/loss=2.35e-01, RTE_valid/accuracy=8.19e-01] RTE:dash_semicolon:[train/loss=9.41e-02, RTE_valid/accuracy=7.87e-01] RTE:more_people:[train/loss=2.23e-02, RTE_valid/accuracy=8.33e-01] model:[train/loss=1.92e-01, train/lr=5.00e-05, valid/glue=8.19e-01]
[2.26 epo]: RTE:[train/loss=2.32e-01, RTE_valid/accuracy=8.12e-01] RTE:dash_semicolon:[train/loss=1.14e-01, RTE_valid/accuracy=7.78e-01] RTE:more_people:[train/loss=1.28e+00, RTE_valid/accuracy=8.33e-01] model:[train/loss=2.13e-01, train/lr=5.00e-05, valid/glue=8.12e-01]
[2.36 epo]: RTE:[train/loss=4.24e-01, RTE_valid/accuracy=7.98e-01] RTE:dash_semicolon:[train/loss=2.84e-01, RTE_valid/accuracy=7.41e-01] RTE:more_people:[train/loss=3.60e-03, RTE_valid

#### Did we improve?

TODO: The final metrics score printed above may be inaccurate... 

Better way to evaluate is by re-evaluating on the checkpoint with the best weights.

In [22]:
from metal.mmtl.metal_model import MetalModel
model.score(payloads_slice[1])

{'RTE/RTE_valid/accuracy': 0.8411552346570397,
 'RTE:dash_semicolon/RTE_valid/accuracy': 0.7870370370370371,
 'RTE:more_people/RTE_valid/accuracy': 0.9166666666666666}